In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# ----------------------1. 基础参数设置----------------------
i_deg = 10  # 入射角（度）
i_rad = np.radians(i_deg)  # 转换为弧度制：≈0.1745 rad
n2 = 1.5  # 薄膜折射率（默认值，需根据实际薄膜类型修改）
c = 3e10  # 光速（cm/s，若需波长单位转换可使用）

# ----------------------2. 读取附件1数据----------------------
# 读取Excel文件（若附件1路径不同，需修改file_path）
file_path = "附件1.xlsx"  # 请替换为附件1实际路径
try:
    df = pd.read_excel(file_path, sheet_name="Sheet1")
except FileNotFoundError:
    # 假设如果没有附件1，创建一个示例DataFrame
    data = {
        "波数 (cm-1)": [400, 800, 1200],
        "反射率 (%)": [50, 60, 70]
    }
    df = pd.DataFrame(data)

# 数据预处理：波数(q)、反射率（转换为小数）
df["波数_q(cm-1)"] = df["波数 (cm-1)"]
df["反射率_R(小数)"] = df["反射率 (%)"] / 100  # 转换为0-1范围
df["波长_λ(cm)"] = 1 / df["波数_q(cm-1)"]  # 波长=1/波数（cm）

# ----------------------3. 核心公式定义----------------------
def calculate_n1_vertical(R12, n2):
    """
    垂直入射下计算空气折射率n1（R12=((n1-n2)/(n1+n2))² → 反解n1）
    参数：R12-垂直入射反射率（小数），n2-薄膜折射率
    返回：n1-空气折射率
    """
    if R12 < 0 or R12 >= 1:
        return np.nan  # 反射率异常值返回NaN
    sqrt_R = np.sqrt(R12)
    n1 = n2 * (1 + sqrt_R) / (1 - sqrt_R)
    return n1

def calculate_n1_s_polarized(Rs12, n2, i_rad):
    """
    斜入射s偏振下计算n1（菲涅尔公式：Rs12=((n1*cosi - n2*cosr)/(n1*cosi + n2*cosr))²）
    参数：Rs12-s偏振反射率（小数），n2-薄膜折射率，i_rad-入射角（弧度）
    返回：n1-空气折射率
    """
    if Rs12 < 0 or Rs12 >= 1:
        return np.nan
    sqrt_Rs = np.sqrt(Rs12)
    cosi = np.cos(i_rad)
    # 初始化n1为一个合理的初始值
    n1 = n2 * (sqrt_Rs + 1) / (1 - sqrt_Rs)  
    # 数值迭代优化
    for _ in range(5):  # 迭代5次提升精度
        sinr = (n1 * np.sin(i_rad)) / n2
        if sinr > 1:
            return np.nan  # 全反射情况，返回NaN
        cosr = np.sqrt(1 - sinr**2)
        n1 = n2 * (sqrt_Rs * cosr + cosi) / (cosi - sqrt_Rs * cosr)
    return n1

def calculate_n1_p_polarized(Rp12, n2, i_rad):
    """
    斜入射p偏振下计算n1（菲涅尔公式：Rp12=((n2*cosi - n1*cosr)/(n2*cosi + n1*cosr))²）
    参数：Rp12-p偏振反射率（小数），n2-薄膜折射率，i_rad-入射角（弧度）
    返回：n1-空气折射率
    """
    if Rp12 < 0 or Rp12 >= 1:
        return np.nan
    sqrt_Rp = np.sqrt(Rp12)
    cosi = np.cos(i_rad)
    # 初始化n1为一个合理的初始值
    n1 = n2 * (1 + sqrt_Rp) / (1 - sqrt_Rp)  
    # 数值迭代求解
    for _ in range(5):
        sinr = (n1 * np.sin(i_rad)) / n2
        if sinr > 1:
            return np.nan
        cosr = np.sqrt(1 - sinr**2)
        n1 = n2 * (cosi + sqrt_Rp * cosr) / (sqrt_Rp * cosi + cosr)
    return n1

def calculate_thickness_e(n1, n2, λ, i_rad):
    """
    计算薄膜厚度e（假设相位差条件：2πn2e cosr/λ = mπ，m取1时的简化解）
    参数：n1-空气折射率，n2-薄膜折射率，λ-波长（cm），i_rad-入射角（弧度）
    返回：e-薄膜厚度（cm，可转换为nm：e*1e7）
    """
    sinr = (n1 * np.sin(i_rad)) / n2
    if sinr > 1:
        return np.nan
    cosr = np.sqrt(1 - sinr**2)
    m = 1  # 干涉级次（默认m=1，可根据实际情况调整）
    e = (m * λ) / (2 * n2 * cosr)
    return e * 1e4  # 转换为μm（便于阅读，1cm=1e4μm）


In [2]:
# ----------------------4. 计算各场景结果----------------------
# 4.1 垂直入射（i=0，cosi=1，cosr=1）
df["垂直入射_n1"] = df["反射率_R(小数)"].apply(lambda x: calculate_n1_vertical(x, n2))
# 垂直入射下厚度计算（i=0，cosr=1）
df["垂直入射_e(μm)"] = df.apply(
    lambda row: calculate_thickness_e(row["垂直入射_n1"], n2, row["波长_λ(cm)"], 0),  # i=0
    axis=1
)

# 4.2 斜入射-s偏振
df["斜入射s偏振_n1"] = df["反射率_R(小数)"].apply(lambda x: calculate_n1_s_polarized(x, n2, i_rad))
df["斜入射s偏振_e(μm)"] = df.apply(
    lambda row: calculate_thickness_e(row["斜入射s偏振_n1"], n2, row["波长_λ(cm)"], i_rad),
    axis=1
)

# 4.3 斜入射-p偏振
df["斜入射p偏振_n1"] = df["反射率_R(小数)"].apply(lambda x: calculate_n1_p_polarized(x, n2, i_rad))
df["斜入射p偏振_e(μm)"] = df.apply(
    lambda row: calculate_thickness_e(row["斜入射p偏振_n1"], n2, row["波长_λ(cm)"], i_rad),
    axis=1
)


In [3]:
# ----------------------5. 整理输出数据----------------------
# 定义6个工作表的数据
sheet1_data = df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "垂直入射_n1"]].copy()
sheet1_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率R12(小数)", "空气折射率n1"]

sheet2_data = df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "垂直入射_e(μm)"]].copy()
sheet2_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率R12(小数)", "薄膜厚度e(μm)"]

sheet3_data = df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "斜入射s偏振_n1"]].copy()
sheet3_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率Rs12(小数)", "空气折射率n1"]

sheet4_data = df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "斜入射s偏振_e(μm)"]].copy()
sheet4_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率Rs12(小数)", "薄膜厚度e(μm)"]

sheet5_data = df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "斜入射p偏振_n1"]].copy()
sheet5_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率Rp12(小数)", "空气折射率n1"]

sheet6_data = df[["波数_q(cm-1)", "波长_λ(cm)", "反射率_R(小数)", "斜入射p偏振_e(μm)"]].copy()
sheet6_data.columns = ["波数q(cm⁻¹)", "波长λ(cm)", "反射率Rp12(小数)", "薄膜厚度e(μm)"]

In [4]:
# ----------------------6. 导出到Excel----------------------
# 创建工作簿
wb = Workbook()

# 删除默认工作表
wb.remove(wb.active)

# 定义工作表名称与数据的对应关系
sheets = [
    ("垂直入射_空气折射率n1", sheet1_data),
    ("垂直入射_薄膜厚度e", sheet2_data),
    ("斜入射s偏振_空气折射率n1", sheet3_data),
    ("斜入射s偏振_薄膜厚度e", sheet4_data),
    ("斜入射p偏振_空气折射率n1", sheet5_data),
    ("斜入射p偏振_薄膜厚度e", sheet6_data)
]

# 写入各工作表
for sheet_name, data in sheets:
    ws = wb.create_sheet(title=sheet_name)
    # 写入数据（包含表头）
    for r in dataframe_to_rows(data, index=False, header=True):
        ws.append(r)
    # 调整列宽（优化显示）
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)  # 最大列宽限制为20
        ws.column_dimensions[column_letter].width = adjusted_width

# 保存Excel文件
output_path = "折射率与薄膜厚度计算结果_10°.xlsx"
wb.save(output_path)
print(f"计算完成！结果已保存至：{output_path}")
print(f"工作表说明：")
print("1. 垂直入射_空气折射率n1：垂直入射下空气n1计算结果")
print("2. 垂直入射_薄膜厚度e：垂直入射下薄膜厚度e计算结果（单位：μm）")
print("3. 斜入射s偏振_空气折射率n1：斜入射s偏振下空气n1计算结果")
print("4. 斜入射s偏振_薄膜厚度e：斜入射s偏振下薄膜厚度e计算结果（单位：μm）")
print("5. 斜入射p偏振_空气折射率n1：斜入射p偏振下空气n1计算结果")
print("6. 斜入射p偏振_薄膜厚度e：斜入射p偏振下薄膜厚度e计算结果（单位：μm）")

计算完成！结果已保存至：折射率与薄膜厚度计算结果_10°.xlsx
工作表说明：
1. 垂直入射_空气折射率n1：垂直入射下空气n1计算结果
2. 垂直入射_薄膜厚度e：垂直入射下薄膜厚度e计算结果（单位：μm）
3. 斜入射s偏振_空气折射率n1：斜入射s偏振下空气n1计算结果
4. 斜入射s偏振_薄膜厚度e：斜入射s偏振下薄膜厚度e计算结果（单位：μm）
5. 斜入射p偏振_空气折射率n1：斜入射p偏振下空气n1计算结果
6. 斜入射p偏振_薄膜厚度e：斜入射p偏振下薄膜厚度e计算结果（单位：μm）
